# <center> GRAMMATICAL ERROR CORRECTION

Notebook de test et d'implémentation de modèles de GEC.
Pour notre GEC nous avons besoin d'un modèle qui prend en entrée un texte et qui renvoie un texte corrigé. Ce genre de modèles s'apparentent à des modèles seq2seq.

## Shared tasks
Plusieurs tâches ont été proposées pour la correction grammaticale durant les années passées:
- [CoNLL-2014](https://www.comp.nus.edu.sg/~nlp/conll14st.html) - ([*paper*](https://www.aclweb.org/anthology/W14-1701.pdf))
- [BEA-2019](https://www.cl.cam.ac.uk/research/nl/bea2019st/) - ([*paper*](https://aclanthology.org/W19-4406.pdf))

En se basant sur ces tâches, il nous est plus aisé de définir quels modèles choisirs et comment les évaluer. **Néanmoins** ces tâches datant de plusieurs années, et le monde de l'intelligence artificiel évoluant rapidement, les modèles utilisés ne sont plus les plus performants (SOTA). Nous allons donc nous baser sur les modèles les plus performants actuellement.

## State Of The Art
Les modèles les plus performants ou les modèles d'état de l'art (State Of The Art) pour les tâches de GEC sont aujourd'hui principalement des modèles transformer [1;2;3;4]. Les modèles transformer sont des modèles de deep learning qui utilisent des mécanismes d'attention pour apprendre des représentations textuelles. Ces modèles sont très performants sur les tâches de NLP (Natural Language Processing) et sont donc très utilisés.

Un modèle réputé pour être très performant est le modèle [T5](https://blog.research.google/2020/02/exploring-transfer-learning-with-t5.html) (Text-to-Text Transfer Transformer) de google. Ce modèle est un modèle transformer (encodeur - décodeur) qui a été pré-entrainé sur un très grand corpus de données. Il ne réalise pas directement la tâche de correction grammaticale mais il est capable de réaliser des tâches de text to text. C'est à dire qu'il prend en entrée un texte et renvoie un texte. Il est donc possible de l'utiliser pour la correction grammaticale, notamment en l'affinant sur un corpus de données de correction grammaticale.

[*1- A Simple Recipe for Multilingual Grammatical Error Correction*](https://arxiv.org/pdf/2106.03830.pdf)
[*2- Grammatical Error Correction: Are We There Yet?*](https://aclanthology.org/2022.coling-1.246/)
[*3- A Comprehensive Survey of Grammatical Error Correction*](https://dl.acm.org/doi/abs/10.1145/3474840)
[*4- Frustratingly Easy System Combination for Grammatical Error Correction*](https://aclanthology.org/2022.naacl-main.143/)
[*5- BTS: Back TranScription for Speech-to-Text Post-Processor using Text-to-Speech-to-Text*](https://aclanthology.org/2021.wat-1.10.pdf)
[*6- LM-Critic: Language Models for Unsupervised Grammatical Error Correction*](https://aclanthology.org/2021.emnlp-main.611.pdf)
[*7- (Almost) Unsupervised Grammatical Error Correction using a Synthetic Comparable Corpus*](https://aclanthology.org/W19-4413.pdf)
[*8- Exploring Effectiveness of GPT-3 in Grammatical Error Correction: A Study on Performance and Controllability in Prompt-Based Method*](https://aclanthology.org/2023.bea-1.18.pdf)
[*9- ChatGPT or Grammarly? Evaluating ChatGPT on Grammatical Error Correction Benchmark*](https://arxiv.org/abs/2303.13648)

## Modèles

Heureusement pour nous, des modèles pré-entrainés existent déjà et sont disponibles sur la librairie [huggingface](https://huggingface.co/). Ces modèles sont des modèles transformer qui ont été pré-entrainés sur des corpus de données de correction grammaticale.

Modèles testés :
- [T5](https://huggingface.co/vennify/t5-base-grammar-correction) (Text-to-Text Transfer Transformer)

Autres modèles :
- [BART](https://huggingface.co/facebook/bart-large-cnn) (Bidirectional and Auto-Regressive Transformers)
- [GPT3](https://huggingface.co/transformers/model_doc/gpt_neo.html) (Generative Pre-trained Transformer 3)
- [BERT](https://huggingface.co/transformers/model_doc/bert.html) (Bidirectional Encoder Representations from Transformers) - [GECwBERT](https://sunilchomal.github.io/GECwBERT/)

Modèle utilisé en pipeline : [T5](https://huggingface.co/vennify/t5-base-grammar-correction)

## Evaluation

Plusieurs metrics existent pour évaluer des modèles GEC:
- F0.5 
- Exact match
- [ERRANT](https://aclanthology.org/P17-1074.pdf)
- [BERTScore](https://huggingface.co/spaces/evaluate-metric/bertscore)
- BLEU
- METEOR
- ...

___
## Imports libraries

In [9]:
!pip3 install bert-score transformers datasets happytransformer

from datasets import load_dataset
import torch, os
from torch.utils.data import DataLoader, Dataset
from datetime import date

from tqdm.notebook import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from happytransformer import HappyTextToText, TTSettings
from evaluate import load

bertscore = load("bertscore")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


___
## Import des données

Nous allons ici définir toutes les fonctions nécessaires à l'import des données. Nous avons besoin de deux datasets:
- Un dataset pour les données en sortie de l'ASR
- Un dataset pour les données de JFLEG (pour l'évaluation)

In [10]:
class ASR_Dataset(Dataset):
    def __init__(self, path, text=None):
        """
        Dataset pour les données en sortie de l'ASR
        :param path: Chemin des données
        :param text: Texte à corriger (Default None, on utilise les données du path. Pour une utilisation du GEC en pipeline alors text est le texte à corriger)
        """
        self.sentences = []
        
        if text is not None:
            self.sentences.append(text.strip())
        else:
            self.path = path
            self.files = os.listdir(path)
            for file in self.files:
                with open(path + '\\' + file, 'r', encoding='utf-8') as file:
                    for line in file:
                        self.sentences.append(line.strip())

    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        transcript = self.sentences[idx]
        return transcript


def load_jfleg_dataset(path='data\\', text=None):
    """
    Dataset pour les données de JFLEG
    :param path: Chemin des données
    :param text: Texte à corriger (Default None, on utilise les données du path. Pour une utilisation du GEC en pipeline alors text est le texte à corriger)
    :return: 
    """
    jfleg_dataset = load_dataset("jfleg", "test", split="test[:10]", cache_dir=path)
    jfleg_Dataloader = DataLoader(jfleg_dataset, batch_size=1)
    
    print("Number of samples:", len(jfleg_dataset))

    sample_meta0 = jfleg_Dataloader.dataset[0]
    print("""
    Transcript n°0 :

    Path audio: {}
    Sentence: {}
    Corrections: {}
    """.format(path, sample_meta0['sentence'], sample_meta0['corrections']))
    
    return jfleg_Dataloader


def process_jfleg(data, model='T5'):
    """
    Fonction de traitement des données JFLEG
    :param data: Données à traiter
    :param model: Modèle à utiliser
    :return: Texte à corriger, checksum, corrections
    """
    sentence = data['sentence']
    corrections = data['corrections']
    
    return sentence[0], 0, corrections


def load_ASR_dataset(path='data\\ASR', text=None, dataset=ASR_Dataset):
    """
    Dataset pour les données en sortie de l'ASR
    :param path: Chemin des données
    :param text: Texte à corriger (Default None, on utilise les données du path. Pour une utilisation du GEC en pipeline alors text est le texte à corriger)
    :param dataset: Classe du dataset à utiliser (Default ASR_Dataset)
    :return: Dataloader ASR
    """
    ASR_Dataset = dataset(path, text)
    ASR_Dataloader = DataLoader(ASR_Dataset, batch_size=1)
    
    print("Number of samples:", len(ASR_Dataset))
    
    sample_meta0 = ASR_Dataset
    print("""
    Transcript n°0 :
    
    Transcript : {}
    """.format(*sample_meta0))
    
    return ASR_Dataloader

def process_ASR(data, model='T5'):
    """
    Fonction de traitement des données ASR
    :param data: Données à traiter
    :param model: Modèle à utiliser
    :return: Texte à corriger, checksum, None
    """
    return data[0], 0, None


def evaluate_jfleg(input, labels, model, tokenizer):
    """
    Fonction d'évaluation des corrections avec JFLEG
    :param input: Texte à corriger
    :param labels: Liste des corrections
    :param model: Modèle à utiliser
    :param tokenizer: Tokenizer à utiliser
    :return: Meilleure correction, score de la correction, score BERT
    """

    results = []
    bert_scores = []
    for correction in labels:
        # Vérifie si la correction est exacte, si oui alors pas besoin de passer par le modèle
        if load('exact_match').compute(references=correction, predictions=[input])['exact_match'] == 1:
            return correction, 5, None
        
        # Calcul du score BERT
        bert_scores.append(bertscore.compute(predictions=[input], references=correction, lang="en"))
        
        # Calcul du score T5
        input_ids = tokenizer.encode("stsb sentence 1: "+input+" sentence 2: "+correction[0], return_tensors="pt").to("cuda")
        stsb_ids = model.generate(input_ids)
        stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
        results.append(float(stsb))
    
    # Récupération de la meilleure correction
    max_arg = results.index(max(results))
    max_score = results[max_arg]
    
    # Récupération du meilleur score BERT
    max_bert = {}
    for scores in bert_scores:
        for key, value in scores.items():
            if key != "hashcode":
                if key not in max_bert:
                    max_bert[key] = value
                else:
                    max_bert[key] = max(max_bert[key], value)
    
    return labels[max_arg], max_score, max_bert
    

___
## Fonction de correction grammaticale

Nous allons ici définir la fonction de correction grammaticale. Cette fonction prend en entrée un texte et renvoie un texte corrigé.
En mode test, cette fonction prend en entrée un chemin de dataset et renvoie un fichier de corrections.

In [11]:
def GEC(text:str = None, model='T5', output_folder='output', dataset='ASR'):
    """
    Fonction de correction grammaticale
    :param text: Texte à corriger (Default None, on utilise les données du chemin du dataset. Pour une utilisation du GEC en pipeline alors text est le texte à corriger)
    :param model: Modèle à utiliser (Default T5)
    :param output_folder: Dossier de sortie (Default output)
    :param dataset: Dataset à utiliser (Default ASR)
    :return: Texte corrigé
    """

    # Création des chemins de sortie
    pipeline = "/pipeline/" if text is not None else "/"
    correction_folder = output_folder + pipeline + model + '_' + dataset
    
    # Chemins de sortie pour les évaluations
    correction_file =f'{correction_folder}/output_GEC.{date.today()}.txt'
    verification = f'{correction_folder}/Verification.{date.today()}.txt'
    vard = f'{correction_folder}/vard.{date.today()}.txt'
    
    os.makedirs(correction_folder, exist_ok=True)
    
    #TQDM loader et fonction de traitement des données
    try:
        function = "load_" + dataset + "_dataset"
        dataloader = eval(function)(text=text)
        dataloader_tqdm = tqdm(dataloader, total=len(dataloader))
        
        process_function = "process_" + dataset
    except Exception as e:
        raise ValueError(f'Unknown dataset {dataset}{e}')
    
    # Création du modèle
    print(f'Generating corrections for {dataset} with {model} in {correction_folder}')
    
    if model == 'T5':
        T5 = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
        args = TTSettings(num_beams=5, min_length=1, max_length=100)
    
    with open(correction_file, 'w', encoding='utf-8') as file:
        file.write('')
    
    # Création des fichiers de vérification et de vard pour JFLEG (pour l'évaluation)
    result = ""
    if dataset == 'jfleg':
        with open(verification, 'w', encoding='utf-8') as file:
                    file.write('')
        with open(vard, 'w', encoding='utf-8') as file:
                    file.write('')
        # Création du modèle pour l'évaluation
        tokenizer = T5Tokenizer.from_pretrained('t5-base', model_max_length=512)
        model_evaluation = T5ForConditionalGeneration.from_pretrained('t5-base').to("cuda")
        mean_evaluation_score = 0
        mean_bert = {}
    
    # Boucle de traitement des données 
    for i, data in enumerate(dataloader_tqdm):
        # Récupération du texte à corriger
        (sentence, out, extra) = eval(process_function)(data, model=model)
        
        # Vérification du checksum
        if out == 1:
            dataloader_tqdm.set_postfix({'status': 'Skipped', 'ID': i})
            continue
            
        # Correction du texte
        try:
            #Passage du transcript dans le modèle
            if model == 'T5':
                result = T5.generate_text("grammar:" + sentence, args=args).text
                with open(correction_file, 'a', encoding='utf-8') as file:
                    file.write(result + '\n')
                if dataset == 'jfleg':
                    with open(vard, 'a', encoding='utf-8') as file:
                        file.write(sentence + '\n')
            elif model == 'BART':
                result = ''
            else:
                raise ValueError(f'Unknown model {model}')
            
            dataloader_tqdm.set_postfix()

        except Exception as e:
            dataloader_tqdm.set_postfix({'status': 'Error', 'ID': i})
            raise e 
        
        # Evaluation du modèle
        if extra is not None:
            best_correction, t5sim, bert_score = evaluate_jfleg(sentence, extra, model_evaluation, tokenizer)
            mean_evaluation_score += t5sim
            
            with open(verification, 'a', encoding='utf-8') as file:
                file.write(best_correction[0] + '\n')
            
            if bert_score is not None:
                for key, value in bert_score.items():
                    val = value[0]
                    if key not in mean_bert:
                        mean_bert[key] = val
                    else:
                        mean_bert[key] += val
        else:
            continue
    
    # Affichage des résultats
    if extra is not None:
        result = f'Mean evaluation score T5: {mean_evaluation_score/len(dataloader)}\n'
        for key, value in mean_bert.items():
            mean_bert[key] /= len(dataloader)
            result += f'Mean {key} score: {mean_bert[key]}\n'
        return result
    
    
    return result

___
## Tests des modèles et des datasets

In [12]:
# # # T5 avec dataset ASR de pipeline
# correction = GEC("I don't have a car but I dreaming it off")
# print(correction)

In [13]:
# # T5 avec dataset ASR
# GEC(model='T5', output_folder='output', dataset='ASR')

In [14]:
# # T5 avec dataset JFLEG
# GEC(model='T5', output_folder='output', dataset='jfleg')

Number of samples: 10

    Transcript n°0 :

    Path audio: data\
    Sentence: New and new technology has been introduced to the society .
    Corrections: ['New technology has been introduced to society .', 'New technology has been introduced into the society .', 'Newer and newer technology has been introduced into society .', 'Newer and newer technology has been introduced to the society .']
    


  0%|          | 0/10 [00:00<?, ?it/s]

Generating corrections for jfleg with T5 in output/T5_jfleg


12/21/2023 19:38:40 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
12/21/2023 19:38:47 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/21/2023 19:38:48 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'Mean evaluation score T5: 4.86\nMean precision score: 0.7689380943775177\nMean recall score: 0.7803486764431\nMean f1 score: 0.7745508134365082\n'

___
## Autres tests

In [15]:
# model_b = T5ForConditionalGeneration.from_pretrained('t5-small').to("cuda")
# stsb_sentence_1 = preprocess_text
# stsb_sentence_2 = output
# input_ids = tokenizer.encode("stsb sentence 1: "+stsb_sentence_1+" sentence 2: "+stsb_sentence_2, return_tensors="pt").to("cuda")
# stsb_ids = model_b.generate(input_ids)
# stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
# print(stsb)
